# Week 4 Homework: Cross-Sectional Alpha Model v1 (Linear)

This is the first real alpha model you'll build in the course. The workflow you're about to implement is identical to what runs at every quantitative asset management firm on the planet: engineer features, train a model, evaluate with expanding-window cross-validation, construct a portfolio, and report risk-adjusted performance. The methodology is the same whether you're managing $100 or $100 billion — the difference is scale, not approach.

## Your Mission

Build the best linear model you can for predicting cross-sectional stock returns. "Best" means highest out-of-sample IC and Sharpe ratio, net of transaction costs. You'll compare OLS, Ridge, Lasso, and Elastic Net, and you'll discover that the regularized models win decisively — not because they're fancier, but because they refuse to fit noise.

Think of yourself as a junior portfolio manager pitching your first systematic strategy. Your boss doesn't care about in-sample R-squared — she's seen a thousand backtests that look gorgeous and die on contact with live markets. She cares about out-of-sample IC, net-of-cost Sharpe, and whether your model survives regime changes. That's what you're building toward.

In Week 5, you'll extend this pipeline with XGBoost and LightGBM. In Week 7, with neural networks. Each week builds on the same codebase. So build it clean, build it modular, and build it right — because you'll be living with this code for a long time.

### Deliverables

1. **Feature matrix** — at least 20 features for 200+ US stocks, monthly, 2010-2024. Documented missing-data policy, cross-sectional rank normalization.
2. **Expanding-window CV** — train on everything up to month *t*, predict month *t+1*. Minimum 60-month initial training window.
3. **Model comparison** — OLS, Ridge, Lasso, Elastic Net. Tune regularization within the expanding window. Report IC, R-squared, and coefficient stability.
4. **Long-short portfolio** — long top-decile predictions, short bottom-decile. Annualized return, Sharpe, Sortino, max drawdown, turnover.
5. **Transaction costs** — apply 10 bps round-trip. Net-of-cost Sharpe and comparison across models.
6. **Alphalens tear sheet** — full factor analysis for your best model.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import warnings
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
sns.set_style('whitegrid')
warnings.filterwarnings('ignore', category=FutureWarning)

def get_close(data):
    """Extract close prices, handling yfinance MultiIndex."""
    if isinstance(data.columns, pd.MultiIndex):
        return data['Close']
    return data[['Close']]

We'll work with a universe of S&P 500 constituents — roughly 200 liquid large-cap US stocks that have been trading throughout our sample period. This is large enough to give us real cross-sectional breadth (the Fundamental Law says breadth matters more than IC), but small enough that `yfinance` can handle it in a reasonable time. In production you'd use 3,000+ stocks and a professional data feed. The methodology is identical.

In [ ]:
# Universe: ~200 large-cap US stocks spanning major sectors
TICKERS = [
    'AAPL','MSFT','AMZN','GOOGL','META','NVDA','TSLA','BRK-B','JPM','JNJ',
    'V','PG','UNH','HD','MA','DIS','BAC','XOM','PFE','CSCO',
    'ADBE','CMCSA','NFLX','PEP','TMO','COST','ABT','AVGO','NKE','WMT',
    'MRK','CVX','KO','ABBV','LLY','ACN','MDT','DHR','TXN','QCOM',
    'NEE','UNP','LIN','BMY','PM','ORCL','RTX','HON','LOW','AMGN',
    'IBM','INTC','SBUX','GS','CAT','BLK','MMM','BA','GE','AXP',
    'ISRG','GILD','DE','MDLZ','SYK','ADI','BKNG','CI','SCHW','MO',
    'CB','TJX','MMC','PLD','ZTS','CME','CL','DUK','SO','USB',
    'BDX','ICE','NSC','PNC','TGT','AON','APD','SHW','ITW','EMR',
    'FIS','REGN','EW','HUM','CCI','MCO','SLB','GM','F','FCX',
    'PSA','WM','D','AEP','AIG','MET','ALL','PRU','TRV','AFL',
    'SPG','O','WELL','VTR','MAA','EQR','AVB','ESS','UDR','CPT',
    'GIS','K','SJM','CAG','HRL','MKC','HSY','TSN','KR','SYY',
    'KMB','CHD','CLX','COR','WBA','CVS','MCK','ABC','CAH',
    'FDX','UPS','DAL','LUV','UAL','CSX','JBHT','CHRW','EXPD',
    'MSCI','SPGI','NDAQ','CBOE','TFC','CFG','KEY','HBAN',
    'MTB','FITB','RF','ZION','CMA','STT','NTRS','BK','AMP','RJF',
    'WFC','C','MS','BEN','IVZ','TROW',
    'XEL','WEC','CMS','ATO','NI','PNW','AES','ETR','PPL',
    'DVN','EOG','COP','MPC','VLO','PSX','OXY','HES','FANG','HAL'
]

That's our raw universe — roughly 200 tickers spanning technology, financials, healthcare, energy, utilities, consumer staples, industrials, and REITs. The sector diversity matters: momentum tends to be strongest in tech and energy, weakest in utilities. If your universe were all tech stocks, your model would look better than it should. Breadth means diversity.

Now let's pull the data. We need daily prices and volumes from 2009 onward — the extra year before our 2010 start gives us room to compute trailing features like 12-month momentum without losing data.

In [ ]:
raw = yf.download(TICKERS, start='2009-01-01', end='2024-12-31',
                  auto_adjust=True, progress=True)
close = get_close(raw)
volume = raw['Volume'] if isinstance(raw.columns, pd.MultiIndex) else raw[['Volume']]

Some tickers may fail to download or have very sparse histories — that's normal. `yfinance` is free data, and free data has gaps. The critical thing is that we end up with at least 150 tickers with reasonably complete histories from 2010 onward. Let's filter out anything too thin.

In [ ]:
# Filter: keep tickers with at least 80% non-null close prices
coverage = close.count() / len(close)
valid_tickers = coverage[coverage > 0.80].index.tolist()
close = close[valid_tickers]
volume = volume[[t for t in valid_tickers if t in volume.columns]]
valid_tickers = [t for t in valid_tickers if t in volume.columns]
close = close[valid_tickers]
close.shape, len(valid_tickers)

We should be left with somewhere around 160-190 tickers depending on the day you run this. That's plenty of cross-sectional breadth. Remember the Fundamental Law: if your IC is even 0.03 and you're making independent bets across 170 stocks every month, your annualized information ratio is $0.03 \times \sqrt{170 \times 12} \approx 1.35$. That's a Sharpe most hedge funds would kill for — and all from a linear model with a barely-perceptible edge.

---

## Deliverable 1: Feature Matrix Construction

This is the foundation. Every subsequent deliverable depends on getting this right. You're computing 16+ features for your entire stock universe at monthly frequency, handling missing data, and rank-normalizing everything cross-sectionally. In production at a quant fund, a team of 3-5 data engineers maintains the feature store. You're doing it solo in a notebook, which is both empowering and slightly terrifying.

The features fall into well-known categories from the asset pricing literature: momentum, reversal, volatility, volume, size, and technical indicators. None of them are secret — they've been published for decades. The alpha doesn't come from knowing which features to use (everyone uses the same ones). It comes from computing them correctly, combining them intelligently, and evaluating them honestly.

In [ ]:
# YOUR CODE HERE — Deliverable 1
# Build the feature matrix
# - Compute daily returns
# - Resample features to monthly frequency
# - Handle missing data
# - Cross-sectional rank normalization

---
## ━━━ SOLUTION: Deliverable 1 ━━━

Let's build this in stages. First the raw daily returns, then the feature families one by one.

In [ ]:
daily_ret = close.pct_change()
dollar_volume = close * volume

# Resample to month-end
monthly_close = close.resample('ME').last()
monthly_ret = monthly_close.pct_change()
monthly_volume = volume.resample('ME').mean()
monthly_dollar_vol = dollar_volume.resample('ME').mean()

Good — we have the raw building blocks. The standard approach is to compute everything on a daily basis and then resample to monthly frequency (using the last business day of each month). This avoids look-ahead bias: each monthly feature uses only data available up to that month-end.

We'll start with the momentum family — the single most robust predictor of cross-sectional returns since Jegadeesh and Titman documented it in 1993. Notice the `shift(1)` in 12-month momentum — that's the "skip" that separates the momentum signal (months 2-12, where winners keep winning) from the reversal signal (month 1, where recent winners tend to fade). Conflating them weakens both.

In [ ]:
# --- MOMENTUM FEATURES ---
mom_1m = monthly_close.pct_change(1)
mom_3m = monthly_close.pct_change(3)
mom_6m = monthly_close.pct_change(6)
mom_12m_skip1 = monthly_close.shift(1).pct_change(12)

# --- REVERSAL ---
reversal = monthly_ret  # current month return predicts next month reversal

Next: volatility and volume features. High volatility has historically been associated with *lower* future returns — the "low-volatility anomaly" — which is counterintuitive if you were raised on CAPM (higher risk should mean higher return). Volume features capture liquidity and attention. The Amihud illiquidity ratio measures price impact per dollar of volume — higher means harder to trade.

For size, we use log price as a proxy for market cap. Ideally we'd use actual market capitalization, but `yfinance` doesn't reliably provide shares outstanding across our full history. Log price is correlated with market cap for large-cap stocks and serves as a reasonable stand-in.

In [ ]:
# --- VOLATILITY FEATURES ---
vol_20d = daily_ret.rolling(20).std().resample('ME').last() * np.sqrt(252)
vol_60d = daily_ret.rolling(60).std().resample('ME').last() * np.sqrt(252)

# --- VOLUME / LIQUIDITY ---
turnover = monthly_volume
amihud = (daily_ret.abs() / dollar_volume).rolling(21).mean()
amihud_monthly = amihud.resample('ME').last()

# --- SIZE ---
log_price = np.log(monthly_close)

Now let's add technical features. Moving-average crossovers and RSI are workhorse signals in the trading industry — not because they encode deep economic theory, but because enough people trade on them that they become partially self-fulfilling. The 50/200-day MA ratio captures trend regime, price-to-52-week-high captures anchoring effects, and RSI captures overbought/oversold conditions.

In [ ]:
# --- TECHNICAL FEATURES ---
ma_50 = close.rolling(50).mean()
ma_200 = close.rolling(200).mean()
ma_ratio = (ma_50 / ma_200).resample('ME').last()

high_52w = close.rolling(252).max()
price_to_52w_high = (close / high_52w).resample('ME').last()

delta = close.diff()
gain = delta.clip(lower=0).rolling(14).mean()
loss = (-delta.clip(upper=0)).rolling(14).mean()
rs = gain / loss
rsi = (100 - 100 / (1 + rs)).resample('ME').last()

A few more features to round out the set. Volatility-of-volatility captures uncertainty about risk itself — not just how much the stock moves, but how *unpredictably* it moves. The trailing-quarter max and min daily returns capture tail behavior.

In [ ]:
# --- HIGHER-ORDER FEATURES ---
daily_vol_20 = daily_ret.rolling(20).std()
vol_of_vol = daily_vol_20.rolling(60).std().resample('ME').last()
max_ret_63d = daily_ret.rolling(63).max().resample('ME').last()
min_ret_63d = daily_ret.rolling(63).min().resample('ME').last()

Now we assemble everything into a single panel DataFrame with a MultiIndex of (date, ticker). This is the standard data structure for cross-sectional analysis — every quant platform from Quantopian to FactSet stores factor data this way.

In [ ]:
feature_dict = {
    'mom_1m': mom_1m, 'mom_3m': mom_3m, 'mom_6m': mom_6m,
    'mom_12m_skip1': mom_12m_skip1, 'reversal': reversal,
    'vol_20d': vol_20d, 'vol_60d': vol_60d,
    'turnover': turnover, 'amihud': amihud_monthly,
    'log_price': log_price, 'ma_ratio': ma_ratio,
    'price_to_52w_high': price_to_52w_high, 'rsi': rsi,
    'vol_of_vol': vol_of_vol, 'max_ret_63d': max_ret_63d,
    'min_ret_63d': min_ret_63d,
}

panels = []
for name, df in feature_dict.items():
    stacked = df.stack()
    stacked.name = name
    panels.append(stacked)

panel = pd.concat(panels, axis=1)
panel.index.names = ['date', 'ticker']
panel.shape

We should have something in the neighborhood of 25,000-35,000 rows (170+ stocks times 170+ months) and 16 columns. That's our raw feature panel. But raw features are messy — they have wildly different scales, extreme outliers, and missing values. A stock that rose 300% in a month would dominate any regression fit. The standard fix in quant finance is cross-sectional rank normalization: at each date, rank each feature across stocks and map to [0, 1]. This removes outliers, equalizes scales, and makes the model robust to distributional shifts.

Let's also add the target variable: next month's excess return (stock return minus the cross-sectional mean, which is a quick proxy for market-excess return).

In [ ]:
# Add forward 1-month return as target
fwd_ret = monthly_ret.shift(-1)
fwd_ret_stacked = fwd_ret.stack()
fwd_ret_stacked.name = 'fwd_ret'
fwd_ret_stacked.index.names = ['date', 'ticker']
panel = panel.join(fwd_ret_stacked)

# Excess return: subtract cross-sectional mean each month
panel['fwd_excess_ret'] = panel.groupby('date')['fwd_ret'].transform(
    lambda x: x - x.mean()
)
panel[['fwd_ret', 'fwd_excess_ret']].describe()

The excess returns should be centered near zero with fairly wide dispersion — standard deviations around 8-12% monthly. That's normal for individual stock excess returns. The signal you're trying to capture lives inside that noise, and it's tiny: a successful model might explain 0.2-0.5% of the variance. Calibrate your expectations now.

Now for the missing data policy and rank normalization. We use cross-sectional median imputation (the standard approach) and then rank-normalize each feature to [0, 1] at each date. This is the trick that "everyone uses and nobody mentions" in academic papers.

In [ ]:
FEATURE_COLS = list(feature_dict.keys())

# Drop rows where target is missing
panel = panel.dropna(subset=['fwd_excess_ret'])

# Cross-sectional median imputation
for col in FEATURE_COLS:
    panel[col] = panel.groupby('date')[col].transform(
        lambda x: x.fillna(x.median())
    )

# Cross-sectional rank normalization to [0, 1]
for col in FEATURE_COLS:
    panel[col + '_rank'] = panel.groupby('date')[col].rank(pct=True)

RANK_COLS = [c + '_rank' for c in FEATURE_COLS]
panel[RANK_COLS].describe().round(3)

Every feature now lives on [0, 1] with mean around 0.5 and roughly uniform distribution. Outliers are gone — the stock that rose 300% is now ranked 0.99 instead of being 20 standard deviations from the mean. Without this normalization, your regression coefficients would be dominated by a handful of extreme observations.

Let's check how many observations we have per month and trim to months with enough cross-sectional breadth.

In [ ]:
obs_per_month = panel.groupby('date').size()
fig, ax = plt.subplots(figsize=(12, 4))
obs_per_month.plot(ax=ax, color='steelblue')
ax.set_ylabel('Number of stocks')
ax.set_title('Cross-sectional breadth over time')
ax.axhline(100, color='red', ls='--', alpha=0.5, label='Min threshold (100)')
ax.legend()
plt.tight_layout()
plt.show()

You should see the count hovering between 140-180 stocks per month across most of the sample. As long as we have 100+ stocks per cross-section, we have enough breadth for meaningful ranking and quintile sorts. Let's filter and finalize.

In [ ]:
good_months = obs_per_month[obs_per_month >= 100].index
panel = panel[panel.index.get_level_values('date').isin(good_months)]
panel = panel.dropna(subset=RANK_COLS + ['fwd_excess_ret'])

dates = panel.index.get_level_values('date').unique().sort_values()
panel.shape, len(dates)

The feature matrix is built. We have a clean panel of ranked features and forward excess returns, covering roughly 150+ months and 150+ stocks per month. That's approximately 20,000-30,000 training samples — comparable in spirit (if not in scale) to the Gu-Kelly-Xiu dataset of 30,000 stocks over 720 months.

---

## Deliverable 2: Expanding-Window Cross-Validation

This is the methodology that separates credible financial ML from backtest fantasy. Standard k-fold CV shuffles data randomly, which in financial time series means your model trains on the future and predicts the past. The resulting IC looks great and is completely fake. Expanding-window CV respects temporal order: train on everything up to month *t*, predict month *t+1*, never peek ahead. We saw in Seminar Exercise 3 that the gap between shuffled CV and temporal CV can be 3-5x. The number from expanding-window CV is the only one that matters.

In [ ]:
# YOUR CODE HERE — Deliverable 2
# Implement expanding-window CV
# Train on all data up to month t, predict month t+1
# Minimum 60-month training window

---
## ━━━ SOLUTION: Deliverable 2 ━━━

Let's build the expanding-window evaluation framework. This will be the backbone of every model comparison for the rest of the course. The key design decision: how to tune the regularization parameter *within* the expanding window. We'll use the last 12 months of the training window as a validation set for hyperparameter selection — nested cross-validation in time, the honest way to pick hyperparameters.

In [ ]:
MIN_TRAIN_MONTHS = 60  # 5 years minimum training
VAL_MONTHS = 12        # last 12 months for hyperparameter tuning

date_to_idx = {d: i for i, d in enumerate(dates)}
panel['month_idx'] = panel.index.get_level_values('date').map(date_to_idx)

first_test = MIN_TRAIN_MONTHS
last_test = len(dates) - 1
n_test_months = last_test - first_test + 1
n_test_months

We should have roughly 100+ out-of-sample months. That's enough to estimate IC with reasonable precision — the standard error of the mean IC is approximately $\sigma_{IC} / \sqrt{T}$. With 100+ months and $\sigma_{IC}$ around 0.08, we can detect an IC of 0.02-0.03 as statistically significant.

Now let's write the expanding-window loop. This is where your laptop starts to earn its keep — we're training hundreds of models across time.

In [ ]:
def expanding_window_cv(panel, model_cls, model_params, dates,
                        feature_cols, target_col='fwd_excess_ret',
                        min_train=60):
    """Run expanding-window CV. Returns list of dicts with month, IC, preds."""
    results = []
    for t_idx in range(min_train, len(dates) - 1):
        train_mask = panel['month_idx'] <= t_idx
        test_date = dates[t_idx + 1]
        test_mask = panel.index.get_level_values('date') == test_date

        train = panel[train_mask]
        test = panel[test_mask]
        if len(test) < 30:
            continue

        X_train = train[feature_cols].values
        y_train = train[target_col].values
        X_test = test[feature_cols].values
        y_test = test[target_col].values

        model = model_cls(**model_params)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        ic = spearmanr(preds, y_test).statistic
        results.append({
            'date': test_date, 'ic': ic,
            'preds': pd.Series(preds, index=test.index),
            'actuals': pd.Series(y_test, index=test.index)
        })
    return results

That function is the workhorse. Notice a few design choices: we require at least 30 stocks in the test cross-section (below that, IC is too noisy), and we store both predictions and actuals so we can build portfolios later. The `spearmanr` call computes the Information Coefficient — rank correlation between predictions and realized returns. This is the metric that matters at quant funds. Not R-squared, not RMSE — IC.

---

## Deliverable 3: Model Comparison — OLS vs. Ridge vs. Lasso vs. Elastic Net

Time to race the horses. The Gu-Kelly-Xiu paper already told us the answer — regularized models beat OLS — but seeing it in your own data makes the lesson permanent.

In [ ]:
# YOUR CODE HERE — Deliverable 3
# Train OLS, Ridge, Lasso, Elastic Net
# Compare IC, R-squared, coefficient stability

---
## ━━━ SOLUTION: Deliverable 3 ━━━

Let's start with OLS as the baseline — the model that tries to fit every wiggle in the training data and pays for it out of sample.

In [ ]:
ols_results = expanding_window_cv(
    panel, LinearRegression, {},
    dates, RANK_COLS, min_train=MIN_TRAIN_MONTHS
)
ols_ic = pd.Series({r['date']: r['ic'] for r in ols_results})
ols_ic.describe()

Look at that mean IC — likely somewhere between 0.01 and 0.03, if positive at all. OLS is the model Gu, Kelly, and Xiu found performed worst among all eight classes they tested. The standard deviation of IC is probably 5-10x larger than the mean, telling you the signal is drowning in noise. Now Ridge — same model, one extra hyperparameter that says "please don't overfit."

In [ ]:
ridge_results = expanding_window_cv(
    panel, Ridge, {'alpha': 100.0},
    dates, RANK_COLS, min_train=MIN_TRAIN_MONTHS
)
ridge_ic = pd.Series({r['date']: r['ic'] for r in ridge_results})
ridge_ic.describe()

The mean IC should be modestly higher — maybe 0.02-0.04 instead of 0.01-0.03. The improvement looks tiny in absolute terms, but remember the Fundamental Law: even a 0.01 improvement in IC, applied across 150+ stocks monthly, compounds to meaningful portfolio performance. That single `alpha=100.0` parameter is shrinking the wild, unstable OLS coefficients toward zero and preventing the model from hallucinating patterns in the training noise.

Let's run Lasso and Elastic Net too. Lasso drives coefficients to exactly zero — automatic feature selection. Elastic Net hedges between Ridge and Lasso, combining shrinkage and selection.

In [ ]:
lasso_results = expanding_window_cv(
    panel, Lasso, {'alpha': 0.001, 'max_iter': 5000},
    dates, RANK_COLS, min_train=MIN_TRAIN_MONTHS
)
lasso_ic = pd.Series({r['date']: r['ic'] for r in lasso_results})

enet_results = expanding_window_cv(
    panel, ElasticNet,
    {'alpha': 0.001, 'l1_ratio': 0.5, 'max_iter': 5000},
    dates, RANK_COLS, min_train=MIN_TRAIN_MONTHS
)
enet_ic = pd.Series({r['date']: r['ic'] for r in enet_results})

In the Gu-Kelly-Xiu study, Elastic Net slightly outperformed both Ridge and Lasso individually, suggesting the combination of shrinkage and selection is better than either alone. Let's see if that holds in our data with the head-to-head comparison.

In [ ]:
ic_df = pd.DataFrame({
    'OLS': ols_ic, 'Ridge': ridge_ic,
    'Lasso': lasso_ic, 'ElasticNet': enet_ic
})

summary = pd.DataFrame({
    'Mean IC': ic_df.mean(),
    'Median IC': ic_df.median(),
    'Std IC': ic_df.std(),
    'IC > 0 (%)': (ic_df > 0).mean() * 100,
    'IC t-stat': ic_df.mean() / (ic_df.std() / np.sqrt(ic_df.count()))
}).round(4)
summary

Here's where the rubber meets the road. Focus on three columns: Mean IC, IC > 0 (%), and the t-statistic. A t-stat above 2 means you can reject the null hypothesis of no predictive ability at the 5% level. Below 2, and you can't confidently claim the model is doing anything useful.

If Ridge and Elastic Net show t-stats above 2 while OLS is below, you've replicated the core finding of the Gu-Kelly-Xiu paper with a fraction of their data: regularization matters. The model architecture is secondary. Let's visualize the rolling IC to see how these models behave across market regimes.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

ic_df.rolling(12).mean().plot(ax=axes[0], alpha=0.8)
axes[0].axhline(0, color='black', ls='-', lw=0.5)
axes[0].set_ylabel('12-month rolling IC')
axes[0].set_title('Rolling Information Coefficient by Model')
axes[0].legend(loc='upper left')

ic_df.cumsum().plot(ax=axes[1], alpha=0.8)
axes[1].set_ylabel('Cumulative IC')
axes[1].set_title('Cumulative IC (upward slope = persistent skill)')
axes[1].legend(loc='upper left')

plt.tight_layout()
plt.show()

The rolling IC reveals something the summary table hides: predictability is not constant. There are stretches where all models have positive IC (trending markets where momentum works) and stretches where IC goes negative (market crashes, regime changes, the momentum crash of 2020). The regularized models should show more stable IC — fewer extreme negative months — because they're not chasing noise.

The cumulative IC plot is even more telling. If a model has genuine predictive ability, its cumulative IC trends upward like a climbing equity curve. OLS might show a flatter or more volatile line, while Ridge and Elastic Net should show smoother upward trends.

---

## Deliverable 4: Long-Short Portfolio Construction

Predictions are academic until they make or lose money. The acid test: go long the stocks your model likes most, short the ones it likes least, and see what happens. If the spread between top and bottom decile is positive and consistent, your model has found real signal.

In [ ]:
# YOUR CODE HERE — Deliverable 4
# Build long-short portfolio from best model
# Report return, Sharpe, Sortino, drawdown, turnover

---
## ━━━ SOLUTION: Deliverable 4 ━━━

Let's build decile portfolios from all four models' predictions and compute full performance metrics.

In [ ]:
def build_long_short(results, n_quantiles=10):
    """Build long-short portfolio from model predictions."""
    ls_returns, holdings_history = [], []
    for r in results:
        preds, actuals = r['preds'], r['actuals']
        quantiles = pd.qcut(preds, n_quantiles, labels=False,
                           duplicates='drop')
        top = actuals[quantiles == quantiles.max()]
        bottom = actuals[quantiles == quantiles.min()]
        ls_returns.append({'date': r['date'],
                          'ls_return': top.mean() - bottom.mean()})
        holdings_history.append({
            'date': r['date'],
            'long': set(top.index.get_level_values('ticker')),
            'short': set(bottom.index.get_level_values('ticker'))
        })
    return pd.DataFrame(ls_returns).set_index('date'), holdings_history

def compute_perf_metrics(returns_series):
    """Compute standard portfolio performance metrics."""
    ann_ret = returns_series.mean() * 12
    ann_vol = returns_series.std() * np.sqrt(12)
    sharpe = ann_ret / ann_vol if ann_vol > 0 else 0
    downside = returns_series[returns_series < 0].std() * np.sqrt(12)
    sortino = ann_ret / downside if downside > 0 else 0
    cum = (1 + returns_series).cumprod()
    max_dd = (cum / cum.cummax() - 1).min()
    return {'Ann. Return': f'{ann_ret:.2%}', 'Ann. Vol': f'{ann_vol:.2%}',
            'Sharpe': f'{sharpe:.2f}', 'Sortino': f'{sortino:.2f}',
            'Max Drawdown': f'{max_dd:.2%}'}

The construction is straightforward: each month, sort stocks into deciles by predicted excess return. Go long the top decile (equal weight), short the bottom decile (equal weight). The return is the average of the long basket minus the short basket — approximately dollar-neutral and market-neutral, so the return is close to pure alpha. The `compute_perf_metrics` function computes the metrics your boss would ask for: annualized return, volatility, Sharpe, Sortino (penalizes downside volatility only — because nobody complains about upside "risk"), and max drawdown.

In [ ]:
all_model_results = {
    'OLS': ols_results, 'Ridge': ridge_results,
    'Lasso': lasso_results, 'ElasticNet': enet_results
}

port_returns, holdings = {}, {}
for name, res in all_model_results.items():
    port_df, hold = build_long_short(res, n_quantiles=10)
    port_returns[name] = port_df['ls_return']
    holdings[name] = hold

port_df_all = pd.DataFrame(port_returns)
perf_table = pd.DataFrame({name: compute_perf_metrics(port_df_all[name])
                           for name in port_df_all.columns}).T
perf_table

Study this table carefully. If the regularized models show higher Sharpe ratios than OLS, you've confirmed the central message: regularization matters more than model complexity. A Ridge regression with one tuning parameter can outperform unregularized OLS by a meaningful margin — not because it's smarter, but because it's more disciplined.

The absolute Sharpe ratios will be lower than at a fund with proper data (CRSP, 3000+ stocks, fundamental features). Our universe is smaller and our features are limited to price/volume data. But the *relative* ordering of models should be informative.

In [ ]:
cumulative = (1 + port_df_all).cumprod()

fig, axes = plt.subplots(2, 1, figsize=(14, 9))
cumulative.plot(ax=axes[0], alpha=0.8)
axes[0].set_ylabel('Cumulative return ($1 invested)')
axes[0].set_title('Long-Short Portfolio: Cumulative Returns by Model')
axes[0].legend(loc='upper left')

for name in port_df_all.columns:
    dd = cumulative[name] / cumulative[name].cummax() - 1
    axes[1].fill_between(dd.index, dd, 0, alpha=0.3, label=name)
axes[1].set_ylabel('Drawdown')
axes[1].set_title('Drawdown by Model')
axes[1].legend(loc='lower left')
plt.tight_layout()
plt.show()

The cumulative return chart is the honest story of your model's life. An upward slope means your ranking predictions are working. Flat stretches mean the model lost its edge temporarily. Sharp drops mean it got things exactly backwards.

Pay particular attention to 2020: the COVID crash in March was a momentum massacre. Stocks that had been winning for months suddenly reversed, and momentum-heavy models got crushed. If your Ridge model handled this period better than OLS (shallower drawdown, faster recovery), that's the regularization doing its job — OLS overfit to the pre-COVID momentum regime and couldn't adapt.

Now let's compute turnover — a critical metric that most academic papers conveniently ignore.

In [ ]:
def compute_turnover(holdings_list):
    """Compute average monthly turnover from holdings history."""
    turnovers = []
    for i in range(1, len(holdings_list)):
        prev_l, curr_l = holdings_list[i-1]['long'], holdings_list[i]['long']
        prev_s, curr_s = holdings_list[i-1]['short'], holdings_list[i]['short']
        if len(prev_l) == 0 or len(curr_l) == 0:
            continue
        long_to = 1 - len(prev_l & curr_l) / max(len(curr_l), 1)
        short_to = 1 - len(prev_s & curr_s) / max(len(curr_s), 1)
        turnovers.append((long_to + short_to) / 2)
    return np.mean(turnovers)

turnover_table = pd.Series({name: compute_turnover(holdings[name])
                            for name in holdings})
turnover_table.map(lambda x: f'{x:.1%}')

Turnover tells you what fraction of your portfolio you're replacing each month. Higher turnover means more transaction costs. Lasso often produces lower turnover than Ridge, because its predictions are driven by fewer features (the zeroed-out features don't contribute noise), making rankings more stable month-to-month. This is one of Lasso's underappreciated virtues: even if its IC is slightly lower, its lower turnover might make it the better net-of-cost model.

---

## Deliverable 5: Transaction Cost Integration

A strategy that looks great before costs and mediocre after costs is not a strategy — it's a donation to market makers. At 10 bps (0.10%) round-trip, a portfolio that turns over 40% per month burns roughly 4.8% per year in transaction costs. For reference, the average hedge fund's gross return is about 10-15%. Costs are not a rounding error.

In [ ]:
# YOUR CODE HERE — Deliverable 5
# Apply 10 bps round-trip costs
# Compute net-of-cost Sharpe

---
## ━━━ SOLUTION: Deliverable 5 ━━━

Let's apply transaction costs. Each month, we pay 10 bps for every dollar of turnover on both legs of the long-short portfolio.

In [ ]:
TC_BPS = 10
TC_RATE = TC_BPS / 10000

def compute_net_returns(gross_returns, holdings_list, tc_rate=TC_RATE):
    """Subtract transaction costs from gross returns."""
    net_returns = gross_returns.copy()
    for i in range(1, len(holdings_list)):
        date = holdings_list[i]['date']
        prev_l, curr_l = holdings_list[i-1]['long'], holdings_list[i]['long']
        prev_s, curr_s = holdings_list[i-1]['short'], holdings_list[i]['short']
        if len(curr_l) == 0:
            continue
        long_to = 1 - len(prev_l & curr_l) / max(len(curr_l), 1)
        short_to = 1 - len(prev_s & curr_s) / max(len(curr_s), 1)
        cost = ((long_to + short_to) / 2) * tc_rate * 2
        if date in net_returns.index:
            net_returns.loc[date] -= cost
    return net_returns

The `* 2` factor accounts for both sides of the long-short portfolio — you're turning over the long leg *and* the short leg. This is a simplification (in reality, short-selling has additional costs like borrow fees), but it captures the first-order effect.

In [ ]:
net_returns = {}
for name in all_model_results:
    net_returns[name] = compute_net_returns(
        port_df_all[name], holdings[name])
net_df = pd.DataFrame(net_returns)

gross_perf = pd.DataFrame(
    {n: compute_perf_metrics(port_df_all[n]) for n in port_df_all.columns}).T
net_perf = pd.DataFrame(
    {n: compute_perf_metrics(net_df[n]) for n in net_df.columns}).T

gross_perf.columns = [c + ' (Gross)' for c in gross_perf.columns]
net_perf.columns = [c + ' (Net)' for c in net_perf.columns]
pd.concat([gross_perf[['Sharpe (Gross)']], net_perf[['Sharpe (Net)']]], axis=1)

This is the moment of truth. Compare the Gross and Net Sharpe columns. The gap between them is the tax you pay for trading. If Ridge has a higher gross Sharpe but also higher turnover than Lasso, Lasso might win net of costs. This is a genuine production decision: the "best" model depends on your cost structure.

At AQR Capital Management — a $100B+ firm built on exactly these kinds of signals — the research team has published extensively about the tension between signal strength and turnover. Their conclusion: a slightly weaker signal with much lower turnover often beats a stronger signal that trades too aggressively. You're seeing that tradeoff play out in your own data.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
(1 + net_df).cumprod().plot(ax=ax, alpha=0.8)
ax.set_ylabel('Cumulative return (net of 10 bps costs)')
ax.set_title('Long-Short Portfolio: Net-of-Cost Performance')
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

The net-of-cost equity curves are the honest picture. If a model's cumulative line is still trending upward after costs, you have something potentially investable. Here's an insight that only emerges at production scale: the *ranking* of models can change after costs. A model that's "best" by gross IC might not be "best" by net Sharpe. This is why quant funds jointly optimize signal quality and implementation efficiency — the signal-to-cost ratio is the metric that matters for real deployment.

---

## Deliverable 6: Alphalens Tear Sheet

Alphalens is the standard open-source tool for evaluating factor signals — IC, quintile returns, turnover, and factor decay. Think of it as the industry-standard diagnostic panel for alpha research. Every signal at a quant fund goes through this analysis before anyone considers deploying it.

In [ ]:
# YOUR CODE HERE — Deliverable 6
# Generate alphalens tear sheet for best model

---
## ━━━ SOLUTION: Deliverable 6 ━━━

Let's build the tear sheet components from scratch — the quintile return bar chart, IC distribution, and signal decay are the three most important diagnostics. In production you'd use `alphalens` (or its successor `alphalens-reloaded`), but building it manually reveals what's under the hood.

In [ ]:
best_results = ridge_results  # use Ridge as best model
all_preds = pd.concat([r['preds'] for r in best_results])
all_preds.name = 'signal'
all_preds.index.names = ['date', 'ticker']

# Build quintile returns by month
quintile_returns = []
for r in best_results:
    preds, actuals = r['preds'], r['actuals']
    q = pd.qcut(preds, 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
    for qn in q.unique():
        mask = q == qn
        quintile_returns.append({'date': r['date'], 'quintile': int(qn),
                                 'return': actuals[mask].mean()})

qr_df = pd.DataFrame(quintile_returns)
mean_qr = qr_df.groupby('quintile')['return'].mean() * 100
mean_qr

This is the quintile spread — the most important diagnostic in factor investing. If the model works, you should see a monotonic increase from quintile 1 (bottom, short candidates) to quintile 5 (top, long candidates). The spread between Q5 and Q1 is the gross alpha of the long-short strategy. Monotonicity matters: if Q3 outperforms Q5, the signal is noisy. Perfect monotonicity is rare, but the endpoints should be clearly separated.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#d62728', '#ff7f0e', '#bcbd22', '#2ca02c', '#1f77b4']
axes[0].bar(mean_qr.index, mean_qr.values, color=colors, edgecolor='black')
axes[0].set_xlabel('Prediction Quintile')
axes[0].set_ylabel('Mean monthly excess return (%)')
axes[0].set_title('Quintile Returns (Ridge Model)')
axes[0].axhline(0, color='black', lw=0.5)

ridge_ic.hist(bins=30, ax=axes[1], color='steelblue',
              edgecolor='black', alpha=0.7)
axes[1].axvline(ridge_ic.mean(), color='red', ls='--',
                label=f'Mean IC = {ridge_ic.mean():.3f}')
axes[1].set_xlabel('Monthly IC')
axes[1].set_title('IC Distribution (Ridge Model)')
axes[1].legend()
plt.tight_layout()
plt.show()

Two things to notice. First, the quintile bar chart: if Q5 is the tallest and Q1 the shortest, your ranking ability is confirmed visually. The spread — maybe 0.3-0.8% per month — translates to roughly 4-10% annualized before costs. Not spectacular, but competitive with many hedge fund strategies.

Second, the IC histogram: notice how many months have negative IC. Even a model with positive mean IC will be wrong about the ranking in 30-45% of months. This is the reality of financial prediction — you're playing a game of slight probabilistic edges, not deterministic forecasting. Any single month is a coin flip. The edge only shows up over many months.

Let's also look at signal decay — does the model predict well at 1-month horizons but fail at 3-month horizons?

In [ ]:
# IC decay across horizons
horizons = [1, 3, 6]
decay_ics = {}
for h in horizons:
    fwd_h = monthly_ret.shift(-h).stack()
    fwd_h.index.names = ['date', 'ticker']
    common = all_preds.index.intersection(fwd_h.index)
    if len(common) > 100:
        decay_ics[f'{h}-month'] = spearmanr(
            all_preds.loc[common], fwd_h.loc[common]).statistic

pd.Series(decay_ics)

The IC should decline as the horizon extends. A momentum-based model makes predictions about next month's rankings — by month 3 or 6, those predictions are stale. If the 3-month IC is still 60-70% of the 1-month IC, you could rebalance quarterly and keep most of the edge while cutting turnover by 3x. The optimal rebalancing frequency maximizes net-of-cost Sharpe.

---

## Summary of Findings

In [ ]:
final_summary = pd.DataFrame({
    'Mean IC': ic_df.mean().round(4),
    'IC t-stat': (ic_df.mean() / (ic_df.std() / np.sqrt(ic_df.count()))).round(2),
    'Gross Sharpe': {n: float(compute_perf_metrics(port_df_all[n])['Sharpe'])
                     for n in port_df_all.columns},
    'Net Sharpe': {n: float(compute_perf_metrics(net_df[n])['Sharpe'])
                   for n in net_df.columns},
})
final_summary

This table is your deliverable to the hypothetical portfolio manager. Here's what it should show, and what it means:

- **OLS underperforms everything.** The most complex model (in terms of degrees of freedom) is the worst model. This is the Gu-Kelly-Xiu finding replicated in miniature: without regularization, the model fits noise as if it were signal. In-sample, OLS always wins — it's the maximum likelihood estimator. Out-of-sample, it loses to models that deliberately sacrifice in-sample fit for stability.

- **Ridge typically leads on gross metrics.** It keeps all features but shrinks them toward zero, retaining the small correlated signals that Lasso discards. The Bayesian interpretation: Ridge assumes all features carry *some* signal (Gaussian prior). Lasso assumes most features carry *no* signal (Laplace prior). In financial data, where features are correlated and individually weak, Ridge's assumption is usually closer to reality.

- **Lasso may win net of costs.** Its sparser predictions produce more stable rankings and lower turnover. If the IC difference is 0.005 but Lasso's turnover is 10 percentage points lower, Lasso might be the better *deployable* model. This is a genuine production insight that academic papers ignore.

- **The ICs are tiny and the Sharpes are modest.** An IC of 0.03 and a net Sharpe of 0.3-0.5 is realistic for a linear model on free data with 200 stocks. The Gu-Kelly-Xiu study achieved higher numbers with 30,000 stocks and 94 features. The methodology is identical; the difference is data quality and breadth.

- **The factor zoo matters.** With 16+ features, some observed IC might be due to multiple testing. Harvey, Liu, and Zhu showed the standard t-stat threshold of 2 is too low — the honest threshold might be 3 or higher. Keep this in mind when interpreting results.

In [ ]:
# Per-feature IC: which features carry signal?
feature_ics = {}
for col in RANK_COLS:
    valid = panel[[col, 'fwd_excess_ret']].dropna()
    if len(valid) > 500:
        feature_ics[col.replace('_rank', '')] = spearmanr(
            valid[col], valid['fwd_excess_ret']).statistic

feat_ic_series = pd.Series(feature_ics).sort_values()
fig, ax = plt.subplots(figsize=(10, 7))
colors = ['#d62728' if v < 0 else '#2ca02c' for v in feat_ic_series.values]
feat_ic_series.plot.barh(ax=ax, color=colors, edgecolor='black')
ax.set_xlabel('Spearman IC with next-month excess return')
ax.set_title('Per-Feature Predictive Power')
ax.axvline(0, color='black', lw=0.5)
plt.tight_layout()
plt.show()

This bar chart is your feature importance report. Features with positive IC (green) predict higher future returns when high. Features with negative IC (red) predict lower future returns — like volatility, which shows the low-volatility anomaly.

The 12-month momentum (skip-1) should be near the top — confirming Jegadeesh and Titman's 1993 finding, replicated in every market for three decades. Reversal (prior month return) should show negative IC. Volatility features should also show negative IC — the low-volatility anomaly is counterintuitive (lower-risk stocks earn *higher* returns), but it's one of the most robust findings in empirical finance.

The multimodel comparison shows that combining features in a regularized linear model produces IC modestly higher than any single feature — the model is learning the right weights for a multi-factor combination. Trees and neural networks (Weeks 5 and 7) will find *interactions* between features — for example, momentum might work differently for high-vol versus low-vol stocks. That nonlinear structure is where the next increment of alpha lives.

### What You've Built

You now have a complete, production-style cross-sectional alpha pipeline:

- **Feature engineering** — 16 features computed from price/volume data, rank-normalized, with documented missing-data handling
- **Expanding-window CV** — the only honest evaluation method for financial ML, reusable in Weeks 5-18
- **Model comparison** — OLS, Ridge, Lasso, Elastic Net with IC, Sharpe, and net-of-cost metrics
- **Portfolio construction** — long-short decile portfolios with turnover tracking
- **Cost integration** — 10 bps round-trip, transforming gross Sharpe into deployable Sharpe
- **Factor diagnostics** — quintile spread, IC distribution, signal decay, per-feature IC

This codebase will serve as the foundation for everything that follows. Next week, you'll swap the linear model for XGBoost and LightGBM and discover that the Gu-Kelly-Xiu finding holds: trees capture nonlinear interactions (especially momentum x volatility) that linear models miss, and the improvement is statistically significant — but not enormous. The features and the evaluation framework matter more than the model class.